In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
def user_index(df_user):  # user groupby df
    user_dct = {row['gPlusUserId']: i for i,row in df_user.iterrows()}
    return user_dct


def place_index(df_place):  # place groupby df
    place_dct = {row['gPlusPlaceId']: i for i,row in df_place.iterrows()}
    return place_dct

place_name = dict(zip(df_places['Name'], df_places['PlaceID']))
user_key = user_index(df_user)
place_key = place_index(df_place)



In [2]:
class MultiRoundRecommendation():
    
    #Initialization
    def __init__(self,user_mat,item_mat,user_key,place_names,place_key,place_reviews):
        
        self.user_feature = user_mat
        self.feature_item = item_mat
        self.userkeys = user_key # userid and index in 
        self.place_reviews = place_reviews 
        self.placenames = place_names  # name and place id 
        self.placekey = place_key   # placeid and index 

    # Single Shot recommendation 
    def get_recommendation(self,user_id):

        idx= self.userkeys[user_id]
        user_review_array= self.user_feature[idx]
        similarities = cosine_similarity(user_review_array,self.feature_item).flatten()
        self.place_reviews['Similarity'] = similarities
        self.place_reviews= self.place_reviews.sort_values("Similarity",ascending = False)
        places_id = self.place_reviews.iloc[0:10,0:1] 
        names = places_id.map(self.placenames)
        return names, self.place_reviews.iloc[0:10,-1],places_id

    def multiround(self,user_id,final_place):
        
        # Final Place : gplusplaceid 
        target_index = self.placekey[final_place]
        target_features = self.feature_item[target_index]

        names,series,places_id = self.get_recommendation(user_id)
        

        
        
        i = 0
        while i < 11:
            i=i+1
            temp_index = places_id.map(place_key)
            temp_feature = []
            # chatgpt  vectorization
            temp_df = self.place_reviews.iloc[temp_index]
            for i in temp_index:
                temp_feature.append(self.feature_item[i])
            
            temp_similarities = cosine_similarity(temp_feature,target_features).flatten() 
            temp_df['Similarity'] = temp_similarities
            temp_df= temp_df.sort_values("Similarity",ascending = False)
            temp_series = temp_df.iloc[0:10,-1]
            places_id = temp_df.iloc[0:10,0:1] 
            names = places_id.map(self.placenames)
            #Breakpoint
            if 1.0 in series:
                print(f"The desired place is found in {i} round.")
                print(names)
                break

            # predictions for round n
            next_feature_index = temp_df.iloc[0,0]
            next_feature = self.feature_item[next_feature_index]
            similarities = cosine_similarity(next_feature,self.feature_item).flatten()
            self.place_reviews['Similarity'] = similarities
            self.place_reviews= self.place_reviews.sort_values("Similarity",ascending = False)
            places_id = self.place_reviews.iloc[1:11,0:1] 
            



















